In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-w7p1/train.csv
/kaggle/input/2023-ml-w7p1/test.csv
/kaggle/input/2023-ml-w7p1/sample_submit.csv


In [2]:
train = pd.read_csv("/kaggle/input/2023-ml-w7p1/train.csv")
test = pd.read_csv("/kaggle/input/2023-ml-w7p1/test.csv")
submit = pd.read_csv("/kaggle/input/2023-ml-w7p1/sample_submit.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   15326 non-null  int64  
 1   enrollee_id             15326 non-null  int64  
 2   city                    15326 non-null  object 
 3   city_development_index  15326 non-null  float64
 4   gender                  11752 non-null  object 
 5   relevent_experience     15326 non-null  object 
 6   enrolled_university     15009 non-null  object 
 7   education_level         14964 non-null  object 
 8   major_discipline        13068 non-null  object 
 9   experience              15276 non-null  object 
 10  company_size            10612 non-null  object 
 11  company_type            10445 non-null  object 
 12  last_new_job            14983 non-null  object 
 13  training_hours          15326 non-null  int64  
 14  target                  15326 non-null

In [4]:
train.head()

,index,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
3,4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
4,5,21651,city_176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1.0


In [5]:
train.drop(['index','enrollee_id'],axis=1,inplace=True)
test.drop(['index','enrollee_id'],axis=1,inplace=True)

In [6]:
x=train.drop(['target'],axis=1)
y=train['target']

In [7]:
x.head()

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36
1,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47
2,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52
3,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8
4,city_176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24


In [8]:
x = x.to_dict('records')
test = test.to_dict('records')

In [9]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False)
x = vec.fit_transform(x)
test = vec.transform(test)

In [10]:
from sklearn.impute import SimpleImputer
im = SimpleImputer()
x=im.fit_transform(x)
test=im.transform(test)

In [11]:
pd.DataFrame(x).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Columns: 193 entries, 0 to 192
dtypes: float64(193)
memory usage: 22.6 MB


In [12]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x=ss.fit_transform(x)
test=ss.transform(test)

In [13]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier()
model.fit(x,y)

KNeighborsClassifier()

In [14]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y,model.predict(x)))

0.827156466135978


In [15]:
submit.columns

Index(['index', 'target'], dtype='object')

In [16]:
submit['target']=model.predict(test)
submit['target'].astype('int')
submit.to_csv("submit.csv",mode='w',header=True,index=False)